In [1]:
import os
import sys
from typing import List, Dict

import lightgbm as lgb
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import train_test_split

# Code 경로 추가
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath(""))))
print(sys.path[-1])

c:\Users\com\Documents\GitHub\level1-classificationinmachinelearning-recsys-06


In [2]:
# 파일 호출
data_path: str = "../../data"
## raw.csv가 없는 경우 실행
# from Code.dataset.merge_all import merge_all
# df = merge_all(data_path)
df: pd.DataFrame = pd.read_csv(os.path.join(data_path, "scaled_eda2.csv"))
submission_df: pd.DataFrame = pd.read_csv(os.path.join(data_path, "test.csv"))  # ID, target 열만 가진 데이터 미리 호출

In [3]:
# 파일 호출
data_path: str = "../../data"
## raw.csv가 없는 경우 실행
# from Code.dataset.merge_all import merge_all
# df = merge_all(data_path)
df: pd.DataFrame = pd.read_csv(os.path.join(data_path, "scaled_eda2.csv"))
submission_df: pd.DataFrame = pd.read_csv(os.path.join(data_path, "test.csv"))  # ID, target 열만 가진 데이터 미리 호출


In [4]:
from imblearn.over_sampling import SMOTE, BorderlineSMOTE, ADASYN, SVMSMOTE

def get_over_sampler(x_train, y_train, strategy='ADASYN', sampling_strategy='auto'):
    if strategy == 'SMOTE':
        sampler = SMOTE(sampling_strategy=sampling_strategy)
    elif strategy == 'BorderlineSMOTE':
        sampler = BorderlineSMOTE(sampling_strategy=sampling_strategy)
    elif strategy == 'ADASYN':
        sampler = ADASYN(sampling_strategy=sampling_strategy)
    elif strategy == 'SVMSMOTE':
        sampler = SVMSMOTE(sampling_strategy=sampling_strategy)
    else:
        raise ValueError(f"Unknown strategy: {strategy}")

    x_resampled, y_resampled = sampler.fit_resample(x_train, y_train)
    return x_resampled, y_resampled

In [26]:
#diff_df = pd.read_csv('eda_df2.csv')
new_feauture = pd.read_csv('new_feature.csv')
new_df = pd.concat([df, new_feauture.drop(columns=['Unnamed: 0','ID'])], axis=1)
#new_df = pd.concat([new_df, diff_df], axis=1)
new_df.columns

Index(['ID', '_type', 'target', 'scaled_log_hashrate',
       'scaled_log_open_interest', 'scaled_log_coinbase_premium_index',
       'scaled_funding_rates', 'scaled_estimated_block_reward',
       'scaled_liquidation_diff', 'scaled_log_total_liquidation',
       'scaled_log_total_taker_volume', 'scaled_utxo_count',
       'scaled_total_transactions_count', 'taker_buy_sell_ratio',
       'moving_avg_scaled_log_total_volume', 'open_interest_diff',
       'network_activity_ratio_diff', 'average_transaction_value_diff',
       'network_load_diff', 'fee_burden_diff', 'market_pressure_diff',
       'liquidation_risk_diff'],
      dtype='object')

In [27]:
new_df2 = new_df.drop(columns=[
    'network_activity_ratio_diff', # 0.437
    #'average_transaction_value_diff',
    #'network_load_diff',
    'fee_burden_diff',
    #'market_pressure_diff', #0.4355
    #'liquidation_risk_diff'
    ],inplace=False)

In [21]:
new_df3 = new_df.drop(columns=[
    'network_activity_ratio_diff', # 0.437
    #'average_transaction_value_diff',
    #'network_load_diff',
    'fee_burden_diff',
    #'market_pressure_diff', #0.4355
    #'liquidation_risk_diff'
    #'open_interest_diff',
    'bsr_diff', #0.417
    'bsr_ma_diff', #0.416
    'tv_diff', #0.414
    'tv_ma_diff', #0.433
    'fr_diff', #0.416 0.434
    'fr_ma_diff' #0.417
    ],inplace=False)

In [28]:
# 제준님이 주신 새로운 피쳐들 사용 (하이퍼 파라미터 적용)

import xgboost as xgb
from imblearn.over_sampling import SMOTE
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split

# _type에 따라 train, test 분리
train_df = new_df2.loc[new_df2["_type"] == "train"].drop(columns=["_type"])
train_df = train_df.ffill()
test_df = new_df2.loc[new_df2["_type"] == "test"].drop(columns=["_type"])

# train_test_split 으로 valid set, train set 분리
x_train, x_valid, y_train, y_valid = train_test_split(
    train_df.drop(["target", "ID"], axis=1),
    train_df["target"].astype(int),
    test_size=0.2,
    random_state=42,
)

# SMOTE를 이용한 오버샘플링
sampling_strategy = {0: 1000, 1: 3544, 2: 3671, 3: 1500}
smote = SMOTE(sampling_strategy=sampling_strategy, random_state=42)
x_train_resampled, y_train_resampled = smote.fit_resample(x_train, y_train)

# XGBoost 모델을 위한 DMatrix로 변환
dtrain = xgb.DMatrix(x_train_resampled, label=y_train_resampled)
dvalid = xgb.DMatrix(x_valid, label=y_valid)

# XGBoost 모델 학습 파라미터
params = {
    "objective": "multi:softprob",  # 다중 클래스 분류
    "num_class": len(y_train_resampled.unique()),  # 클래스 개수
    "eval_metric": "mlogloss",  # 손실 함수 (멀티클래스 로지스틱 손실)
    "max_depth": 7,
    "learning_rate": 0.01,
    "subsample": 0.8,
    "colsample_bytree": 0.8,
    "seed": 42
}

# 교차 검증 설정 (5-fold)
cv_results = xgb.cv(
    params=params,
    dtrain=dtrain,
    num_boost_round=200,
    nfold=5,
    early_stopping_rounds=10,
    metrics="mlogloss",
    as_pandas=True,
    seed=42
)

# 최적의 부스팅 라운드 수 확인
best_num_boost_rounds = len(cv_results)

# 교차 검증 결과 출력
print("최적 부스팅 라운드 수:", best_num_boost_rounds)
print(cv_results.tail())

# 최적의 부스팅 라운드로 모델 학습
xgb_model = xgb.train(
    params=params,
    dtrain=dtrain,
    num_boost_round=best_num_boost_rounds,
    evals=[(dtrain, "train"), (dvalid, "eval")],
    early_stopping_rounds=10
)

# 예측 (훈련 데이터셋)
y_train_pred_prob = xgb_model.predict(dtrain)
y_train_pred = y_train_pred_prob.argmax(axis=1)

# 예측 (검증 데이터셋)
y_valid_pred_prob = xgb_model.predict(dvalid)
y_valid_pred = y_valid_pred_prob.argmax(axis=1)

# 성능 평가 (훈련 데이터셋)
train_accuracy = accuracy_score(y_train_resampled, y_train_pred)
train_classification_rep = classification_report(y_train_resampled, y_train_pred)

# 성능 평가 (검증 데이터셋)
valid_accuracy = accuracy_score(y_valid, y_valid_pred)
valid_classification_rep = classification_report(y_valid, y_valid_pred)

# 결과 출력
print(f"훈련 정확도: {train_accuracy}")
print(train_classification_rep)
print(f"검증 정확도: {valid_accuracy}")
print(valid_classification_rep)


최적 부스팅 라운드 수: 200
     train-mlogloss-mean  train-mlogloss-std  test-mlogloss-mean  \
195             0.972077            0.003553            1.163700   
196             0.970873            0.003565            1.163238   
197             0.969728            0.003548            1.162849   
198             0.968550            0.003568            1.162394   
199             0.967391            0.003608            1.161960   

     test-mlogloss-std  
195           0.008732  
196           0.008771  
197           0.008804  
198           0.008785  
199           0.008807  
[0]	train-mlogloss:1.38253	eval-mlogloss:1.38359
[1]	train-mlogloss:1.37885	eval-mlogloss:1.38078
[2]	train-mlogloss:1.37524	eval-mlogloss:1.37818
[3]	train-mlogloss:1.37162	eval-mlogloss:1.37551
[4]	train-mlogloss:1.36797	eval-mlogloss:1.37272
[5]	train-mlogloss:1.36441	eval-mlogloss:1.37005
[6]	train-mlogloss:1.36075	eval-mlogloss:1.36755
[7]	train-mlogloss:1.35740	eval-mlogloss:1.36512
[8]	train-mlogloss:1.35395	eval

In [16]:
import plotly.express as px

# Feature importance 평가
importance = xgb_model.get_score(importance_type='weight')
importance_df = pd.DataFrame({
    'Feature': [k for k in importance.keys()],
    'Importance': importance.values()
}).sort_values(by='Importance', ascending=False)

# 시각화
fig = px.bar(importance_df, x='Feature', y='Importance', title='Feature Importance')
fig.show()

In [17]:
# 테스트 데이터셋을 DMatrix로 변환
dtest = xgb.DMatrix(test_df.drop(["target", "ID"], axis=1))

# 테스트 데이터에 대한 예측 수행
y_test_pred_prob = xgb_model.predict(dtest)

# 다중 클래스 예측일 경우 가장 높은 확률을 가진 클래스를 예측값으로 변환
y_test_pred = y_test_pred_prob.argmax(axis=1)

# 예측 결과 출력
y_test_pred

submission_df = submission_df.assign(target=pd.DataFrame(y_test_pred))
submission_df['target'].value_counts()

target
2    1815
1     789
3     128
0      60
Name: count, dtype: int64